In [14]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:407: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,8.818490,False,17,1,42.0
1,8.141671,False,29,1,38.0
2,5.948072,True,38,1,38.0
3,8.838332,True,27,1,39.0
4,9.259415,True,28,1,38.0


In [15]:
df['is_male'].value_counts()

True     5158
False    4842
Name: is_male, dtype: int64

In [16]:
df = df.dropna()
df = shuffle(df, random_state=2)
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)
df.head()


,weight_pounds,is_male,mother_age,plurality,gestation_weeks
757,7.374463,True,23,1,39.0
6544,8.375361,True,19,1,38.0
3190,6.239082,True,28,1,40.0
2064,8.437091,False,32,1,40.0
4066,7.374463,False,17,1,40.0


In [17]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [18]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [19]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [20]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6665 samples, validate on 741 samples
Epoch 1/10
6665/6665 [==============================] - 2s 369us/sample - loss: 3.7764 - mae: 1.1921 - mse: 3.7764 - val_loss: 1.3362 - val_mae: 0.9113 - val_mse: 1.3362
Epoch 2/10
6665/6665 [==============================] - 1s 203us/sample - loss: 1.3504 - mae: 0.9123 - mse: 1.3504 - val_loss: 1.4121 - val_mae: 0.9267 - val_mse: 1.4121
Epoch 3/10
6665/6665 [==============================] - 2s 239us/sample - loss: 1.3223 - mae: 0.9053 - mse: 1.3223 - val_loss: 1.6021 - val_mae: 1.0139 - val_mse: 1.6021
Epoch 4/10
6665/6665 [==============================] - 1s 173us/sample - loss: 1.3094 - mae: 0.8990 - mse: 1.3094 - val_loss: 1.5048 - val_mae: 0.9606 - val_mse: 1.5048
Epoch 5/10
6665/6665 [==============================] - 1s 169us/sample - loss: 1.2932 - mae: 0.8918 - mse: 1.2932 - val_loss: 1.2278 - val_mae: 0.8647 - val_mse: 1.2278
Epoch 6/10
6665/6665 [==============================] - 1s 187us/sample - loss: 1.2760 - mae: 0.8884 - 

In [ ]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [ ]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()